In [1]:
# McKinley Redden

In [3]:
# import libraries
import pandas as pd
import numpy as np
import psycopg2
import pymssql

In [4]:
# connect to the postgres database
pg_conn = psycopg2.connect(
    database='iwdm',
    user='data5330',
    password='data5330',
    host='data5330.ctcggyeoeoua.us-east-2.rds.amazonaws.com',
    port='5432'
)
pg_cursor = pg_conn.cursor()

In [4]:
# create the customer table
query = '''CREATE TABLE IF NOT EXISTS customer (
            customerid INT PRIMARY KEY,
            customerfname VARCHAR(50) NOT NULL,
            customerlname VARCHAR(50) NOT NULL,
            customerage SMALLINT,
            customertype VARCHAR(6),
            customersince SMALLINT,
            customerincome INT,
            householdsize SMALLINT,
            modecolor VARCHAR(10)
        );'''
pg_cursor.execute(query)
pg_conn.commit()

In [5]:
# create the station table
query = '''CREATE TABLE IF NOT EXISTS station (
            weatherstationcode INT PRIMARY KEY,
            station VARCHAR(15),
            stationname VARCHAR(60),
            latitude NUMERIC(8,5),
            longitude NUMERIC(8,5),
            elevation NUMERIC(5,1)
        );'''
pg_cursor.execute(query)
pg_conn.commit()

In [6]:
# create the stadium table
query = '''CREATE TABLE IF NOT EXISTS stadium (
            stadiumid SERIAL PRIMARY KEY,
            stadiumname VARCHAR(40) NOT NULL,
            stadiumlocation VARCHAR(25),
            stadiumopen SMALLINT,
            stadiumclose SMALLINT,
            stadiumtype VARCHAR(15),
            stadiumaddress VARCHAR(100),
            stadiumweatherstationcode INT,
            FOREIGN KEY (stadiumweatherstationcode) REFERENCES station(weatherstationcode),
            stadiumweathertype VARCHAR(10),
            stadiumcapacity INT,
            stadiumsurface VARCHAR(25)
        );'''
pg_cursor.execute(query)
pg_conn.commit()            

In [7]:
# create the team table
query = '''CREATE TABLE IF NOT EXISTS team (
            teamid VARCHAR(4) PRIMARY KEY,
            teamname VARCHAR(30) NOT NULL,
            teamnameshort VARCHAR(15),
            teamidpfr VARCHAR(3),
            teamconference CHAR(3),
            teamdivision VARCHAR(15),
            teamconferencepre2002 CHAR(3),
            teamdivisionpre2002 VARCHAR(15)
        );'''
pg_cursor.execute(query)
pg_conn.commit()

In [8]:
# create the game table
query = '''CREATE TABLE IF NOT EXISTS game (
            gameid VARCHAR(50) PRIMARY KEY,
            scheduledate DATE,
            scheduleseason SMALLINT,
            scheduleweek VARCHAR(20),
            scheduleplayoff BOOLEAN,
            hometeamid VARCHAR(3),
            FOREIGN KEY (hometeamid) REFERENCES team(teamid),
            homescore SMALLINT,
            awayscore SMALLINT,
            awayteamid VARCHAR(3),
            FOREIGN KEY (awayteamid) REFERENCES team(teamid),
            teamfavoriteid VARCHAR(4),
            FOREIGN KEY (teamfavoriteid) REFERENCES team(teamid),
            spreadfavorite NUMERIC(3,1),
            overunderline NUMERIC(3,1),
            stadiumid INT,
            FOREIGN KEY (stadiumid) REFERENCES stadium(stadiumid),
            stadiumneutral BOOLEAN,
            weathertemperature SMALLINT,
            weatherwindMPH SMALLINT,
            weatherhumidity SMALLINT,
            weatherdetail VARCHAR(25),
            winner_ou VARCHAR(5),
            winner_line CHAR(4)
        );'''
pg_cursor.execute(query)
pg_conn.commit()

In [9]:
# create the betlog table
query = '''CREATE TABLE IF NOT EXISTS betlog (
            betid INT PRIMARY KEY,
            customerid INT NOT NULL,
            FOREIGN KEY (customerid) REFERENCES customer(customerid),
            gameid VARCHAR(50),
            FOREIGN KEY (gameid) REFERENCES game(gameid),
            betamount SMALLINT,
            beton VARCHAR(50),
            result VARCHAR(4),
            commissionvalue SMALLINT
        );'''
pg_cursor.execute(query)
pg_conn.commit()

In [10]:
# connect to the SQL Server database
ss_conn = pymssql.connect(
    server='stairwaytoheaven.usu.edu',
    database='ironwill',
    user='5330user',
    password='pipelinesnow')
ss_cursor = ss_conn.cursor()

In [11]:
# populate the customer table
customer_df = pd.read_sql(('''
    SELECT *
    FROM customer_table
    ;'''), ss_conn)
for index, row in customer_df.iterrows():
    name_split = row.iloc[1].split(' ')
    pg_cursor.execute(('''
        INSERT INTO customer (customerid, customerfname, customerlname, customerage, customertype, customersince, customerincome, householdsize, modecolor)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ;'''), (row.iloc[0], name_split[0], name_split[1], row.iloc[2], row.iloc[3], row.iloc[4], row.iloc[5], row.iloc[6], row.iloc[7]))
    pg_conn.commit()

C:\Users\redde\AppData\Local\Temp\ipykernel_3964\782671579.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_df = pd.read_sql(('''


In [12]:
# populate the station table
stadium_df = pd.read_csv('nfl_stadiums.csv', encoding='ISO-8859-1')
for index, row in stadium_df.iterrows():
    try:
        stationcode = int(row.iloc[6])
    except:
        stationcode = 0
    if stationcode != 0:
        pg_cursor.execute(('''
            INSERT INTO station (weatherstationcode, station, stationname, latitude, longitude, elevation)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (weatherstationcode) DO NOTHING
        ;'''), (stationcode, row.iloc[10], row.iloc[11], row.iloc[12], row.iloc[13], row.iloc[14]))
        pg_conn.commit()

In [13]:
# populate stadium table
stadium_df = pd.read_csv('nfl_stadiums.csv', encoding='ISO-8859-1')
stadium_df = stadium_df.fillna(value='')
for index, row in stadium_df.iterrows():
    try:
        stadiumopen = int(row.iloc[2])
    except:
        stadiumopen = None
    try:
        stadiumclose = int(row.iloc[3])
    except:
        stadiumclose = None
    try:
        stationcode = int(row.iloc[6])
    except:
        stationcode = None
    try:
        capacity_str = row.iloc[8].replace(',', '')
        capacity = int(capacity_str)
    except:
        capacity = None
    pg_cursor.execute(('''
        INSERT INTO stadium (stadiumname, stadiumlocation, stadiumopen, stadiumclose, stadiumtype, stadiumaddress, stadiumweatherstationcode, stadiumweathertype, stadiumcapacity, stadiumsurface)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ;'''), (row.iloc[0], row.iloc[1], stadiumopen, stadiumclose, row.iloc[4], row.iloc[5], stationcode, row.iloc[7], capacity, row.iloc[9]))
    pg_conn.commit()

In [14]:
# populate team table
team_df = pd.read_csv('nfl_teams.csv')
# drop the duplicate teams
teams_to_remove = ['Baltimore Colts', 'Tennessee Oilers', 'Washington Football Team', 'Washington Redskins', 'Los Angeles Raiders', 
                   'Boston Patriots', 'Phoenix Cardinals', 'St. Louis Cardinals', 'Oakland Raiders', 'San Diego Chargers', 'St. Louis Rams', 
                   'Houston Oilers']
team_df = team_df[~team_df['team_name'].isin(teams_to_remove)]
for index, row in team_df.iterrows():
    pg_cursor.execute(('''
        INSERT INTO team (teamid, teamname, teamnameshort, teamidpfr, teamconference, teamdivision, teamconferencepre2002, teamdivisionpre2002)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ;'''), (row.iloc[2], row.iloc[0], row.iloc[1], row.iloc[3], row.iloc[4], row.iloc[5], row.iloc[6], row.iloc[7]))
    pg_conn.commit()

In [15]:
# populate the game table
game_df = pd.read_csv('spread_scores-2.csv')
# get the team and stadium ids
team_lookup = pd.read_sql('SELECT teamname, teamid FROM team;', pg_conn).set_index('teamname')['teamid'].to_dict()
stadium_lookup = pd.read_sql('SELECT stadiumname, stadiumid FROM stadium;', pg_conn).set_index('stadiumname')['stadiumid'].to_dict()
# insert pick into the team table to avoid foreign key issues
if 'PICK' not in team_lookup.values():
    pg_cursor.execute('''INSERT INTO team (teamid, teamname)
    VALUES ('PICK', 'Pick’em');''')
    pg_conn.commit()
for index, row in game_df.iterrows():
    # only include games starting in the 2015 season
    if row.iloc[1] >= 2015:
        # replace Nan with None in any int fields
        try:
            scheduleseason = int(row.iloc[1])
        except:
            scheduleseason = None
        def parse_int(value):
            try:
                return int(value)
            except:
                return None
        homescore = parse_int(row.iloc[5])
        awayscore = parse_int(row.iloc[6])
        weathertemperature = parse_int(row.iloc[13])
        weatherwindmph = parse_int(row.iloc[14])
        weatherhumidity = parse_int(row.iloc[15])
        # add a 0 in front of the week if necessary
        if len(row.iloc[2]) < 2:
            week = '0' + row.iloc[2]
        else:
            week = row.iloc[2]
        # get the teamid
        hometeam = team_lookup.get(row['team_home'])
        awayteam = team_lookup.get(row['team_away'])
        stadiumid = stadium_lookup.get(row['stadium'])
        if hometeam is None or awayteam is None:
            continue
        # put together the gameid
        gameid = str(row.iloc[1]) + week + '-' + hometeam + '-' + awayteam
        # figure out who wins the over-under
        totalscore = int(row.iloc[5]) + int(row.iloc[6])
        if totalscore > int(row.iloc[10]):
            winner_ou = 'over'
        elif totalscore < int(row.iloc[10]):
            winner_ou = 'under'
        else:
            winner_ou = 'push'
        # figure out who wins the betting line
        if homescore is not None:
            if awayscore is not None:
                if hometeam == row.iloc[8]:
                    difference = homescore - awayscore
                    if difference >= 0:
                        if difference > abs(row.iloc[9]):
                            winner_line = 'home'
                        elif difference < abs(row.iloc[9]):
                            winner_line = 'away'
                        else:
                            winner_line = 'push'
                    else:
                        winner_line = 'away'
                elif awayteam == row.iloc[8]:
                    difference = awayscore - homescore
                    if difference >= 0:
                        if difference > abs(row.iloc[9]):
                            winner_line = 'away'
                        elif difference < abs(row.iloc[9]):
                            winner_line = 'home'
                        else:
                            winner_line = 'push'
                    else:
                        winner_line = 'home'
                elif row.iloc[8] == 'PICK':
                    if homescore > awayscore:
                        winner_line = 'home'
                    elif homescore < awayscore:
                        winner_line = 'away'
                    else:
                        winner_line = 'push'
        try:
            pg_cursor.execute(('''
                INSERT INTO game (gameid, scheduledate, scheduleseason, scheduleweek, scheduleplayoff, hometeamid, homescore, awayscore, awayteamid, teamfavoriteid, 
                spreadfavorite, overunderline, stadiumid, stadiumneutral, weathertemperature, weatherwindmph, weatherhumidity, weatherdetail, winner_ou, winner_line)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (gameid) DO NOTHING
            ;'''), (gameid, row.iloc[0], row.iloc[1], row.iloc[2], row.iloc[3], hometeam, homescore, awayscore, awayteam, row.iloc[8], row.iloc[9], row.iloc[10], stadiumid, row.iloc[12], 
                    weathertemperature, weatherwindmph, weatherhumidity, row.iloc[16], winner_ou, winner_line))
            pg_conn.commit()
        except:
            print('Error with:', gameid, row.iloc[0], row.iloc[1], row.iloc[2], row.iloc[3], hometeam, homescore, awayscore, awayteam, row.iloc[8], row.iloc[9], row.iloc[10], stadiumid, row.iloc[12], 
                    weathertemperature, weatherwindmph, weatherhumidity, row.iloc[16], winner_ou, winner_line)
            pg_conn.rollback()

C:\Users\redde\AppData\Local\Temp\ipykernel_3964\2112375853.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  team_lookup = pd.read_sql('SELECT teamname, teamid FROM team;', pg_conn).set_index('teamname')['teamid'].to_dict()
C:\Users\redde\AppData\Local\Temp\ipykernel_3964\2112375853.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stadium_lookup = pd.read_sql('SELECT stadiumname, stadiumid FROM stadium;', pg_conn).set_index('stadiumname')['stadiumid'].to_dict()


In [16]:
# PLEASE READ THIS WHEN GRADING
print('I tried to populate the betlog table multiple times, however it shut down after a while of running. Due to computer and time restraints, I decided to continue the project with only about half of the data populated. This was enough to show that my queries do run as expected, they just might have slightly difference results due to the missing data.')

I tried to populate the betlog table multiple times, however it shut down after a while of running. Due to computer and time restraints, I decided to continue the project with only about half of the data populated. This was enough to show that my queries do run as expected, they just might have slightly difference results due to the missing data.


In [ ]:
# populate the betlog table
betlog_df = pd.read_sql(('''
    SELECT *
    FROM betlog
    ;'''), ss_conn)
betlog_df['game_id'] = betlog_df['game_id'].fillna('')
betlog_df['game_id'] = betlog_df['game_id'].str.replace('LV', 'LVR')
betlog_df['game_id'] = betlog_df['game_id'].str.replace('JAC', 'JAX')

game_df = pd.read_sql('SELECT gameid, winner_ou, winner_line, hometeamid, awayteamid FROM game;', pg_conn)
team_df = pd.read_sql('SELECT teamid, teamname FROM team;', pg_conn)
game_dict = game_df.set_index('gameid').to_dict('index')
team_dict = team_df.set_index('teamname')['teamid'].to_dict()

def calculate_commission(betamount):
    if betamount <= 1000:
        return betamount * 0.10
    elif betamount <= 5000:
        return 1000 * 0.10 + (betamount - 1000) * 0.08
    else:
        return 1000 * 0.10 + 4000 * 0.08 + (betamount - 5000) * 0.06

def determine_result(row):
    game_id = row['game_id']
    bet_on = row['bet_on']

    game = game_dict.get(game_id)
    if not game:
        return None

    if bet_on in ['over', 'under']:
        if bet_on == game['winner_ou']:
            return 'win'
        elif game['winner_ou'] == 'push':
            return 'push'
        else:
            return 'loss'
    else:
        team_id = team_dict.get(bet_on)
        if not team_id:
            return None
        if team_id == game['hometeamid']:
            if game['winner_line'] == 'home':
                return 'win'
            elif game['winner_line'] == 'away':
                return 'loss'
            else:
                return 'push'
        elif team_id == game['awayteamid']:
            if game['winner_line'] == 'away':
                return 'win'
            elif game['winner_line'] == 'home':
                return 'loss'
            else:
                return 'push'
        else:
            return None

for index, row in betlog_df.iterrows():
    commission = calculate_commission(row.iloc[3])
    result = determine_result(row)
    try:
        pg_cursor.execute(('''
            INSERT INTO betlog (betid, customerid, gameid, betamount, beton, result, commissionvalue)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (betid) DO NOTHING
        ;'''), (row.iloc[0], row.iloc[1], row.iloc[2], row.iloc[3], row.iloc[4], result, commission))
        pg_conn.commit()
    except:
        pg_conn.rollback()

C:\Users\redde\AppData\Local\Temp\ipykernel_3964\2759680696.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  betlog_df = pd.read_sql(('''
C:\Users\redde\AppData\Local\Temp\ipykernel_3964\2759680696.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  game_df = pd.read_sql('SELECT gameid, winner_ou, winner_line, hometeamid, awayteamid FROM game;', pg_conn)
C:\Users\redde\AppData\Local\Temp\ipykernel_3964\2759680696.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  team_df = pd.read_sql('SELECT teamid, teamname FROM tea

In [1]:
# Test Queries

In [5]:
# 1 - betlog table
bl_df = pd.read_sql('''
    select *
    from betlog
    limit 5;''', pg_conn)
print(bl_df)

C:\Users\redde\AppData\Local\Temp\ipykernel_15296\1049067707.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bl_df = pd.read_sql('''


   betid  customerid         gameid  betamount               beton result  \
0      1           1  202301-KC-DET       8500       Detroit Lions    win   
1      2           7  202301-KC-DET        350               under    win   
2      3           8  202301-KC-DET      10000  Kansas City Chiefs   loss   
3      4          23  202301-KC-DET        200               under    win   
4      5          24  202301-KC-DET        400                over   loss   

   commissionvalue  
0              630  
1               35  
2              720  
3               20  
4               40  


In [6]:
# 1 - customer table
c_df = pd.read_sql('''
    select *
    from customer
    limit 5;''', pg_conn)
print(c_df)

   customerid customerfname customerlname  customerage customertype  \
0           1      Coraline        Flores           23       online   
1           2       Presley         Ortiz           41       online   
2           3          Reid        Knight           40       online   
3           4      Clarissa      Chandler           43        local   
4           5         Isaac        Vaughn           23        phone   

   customersince  customerincome  householdsize modecolor  
0           2022          116000              1       red  
1           2021          111000              1      blue  
2           2023           55000              1     green  
3           2023           32000              2    orange  
4           2023           46000              1      blue  


C:\Users\redde\AppData\Local\Temp\ipykernel_15296\366571818.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  c_df = pd.read_sql('''


In [7]:
# 1 - game table
g_df = pd.read_sql('''
    select *
    from game
    limit 5;''', pg_conn)
print(g_df)

           gameid scheduledate  scheduleseason scheduleweek  scheduleplayoff  \
0   201501-NE-PIT   2015-09-10            2015            1            False   
1   201501-ARI-NO   2015-09-13            2015            1            False   
2  201501-BUF-IND   2015-09-13            2015            1            False   
3   201501-CHI-GB   2015-09-13            2015            1            False   
4  201501-DAL-NYG   2015-09-13            2015            1            False   

  hometeamid  homescore  awayscore awayteamid teamfavoriteid  spreadfavorite  \
0         NE         28         21        PIT             NE            -7.0   
1        ARI         31         19         NO            ARI            -2.5   
2        BUF         27         14        IND            IND            -1.0   
3        CHI         23         31         GB             GB            -6.5   
4        DAL         27         26        NYG            DAL            -7.0   

   overunderline  stadiumid  stadiumne

C:\Users\redde\AppData\Local\Temp\ipykernel_15296\1101504056.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  g_df = pd.read_sql('''


In [8]:
# 1 - stadium table
sd_df = pd.read_sql('''
    select *
    from stadium
    limit 5;''', pg_conn)
print(sd_df)

   stadiumid        stadiumname   stadiumlocation  stadiumopen stadiumclose  \
0          1   Acrisure Stadium    Pittsburgh, PA       2001.0         None   
1          2         Alamo Dome   San Antonio, TX          NaN         None   
2          3  Allegiant Stadium      Paradise, NV       2020.0         None   
3          4      Allianz Arena   Munich, Germany          NaN         None   
4          5     Alltel Stadium  Jacksonville, FL          NaN         None   

  stadiumtype                            stadiumaddress  \
0     outdoor  100 Art Rooney Ave, Pittsburgh, PA 15212   
1      indoor     100 Montana St, San Antonio, TX 78203   
2      indoor                                             
3     outdoor                                             
4                                                         

   stadiumweatherstationcode stadiumweathertype  stadiumcapacity  \
0                    15212.0               cold          65500.0   
1                    78203.0      

C:\Users\redde\AppData\Local\Temp\ipykernel_15296\3844361489.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sd_df = pd.read_sql('''


In [9]:
# 1 - station table
st_df = pd.read_sql('''
    select *
    from station
    limit 5;''', pg_conn)
print(st_df)

   weatherstationcode      station                stationname  latitude  \
0               15212  USW00094823     PITTSBURGH ASOS, PA US   40.4846   
1               78203          NaN                        NaN       NaN   
2                2467          NaN                        NaN       NaN   
3               92806          NaN                        NaN       NaN   
4               64129  US1MOJC0028  KANSAS CITY 5.1 SE, MO US   39.0692   

   longitude  elevation  
0   -80.2144      366.7  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4   -94.4871      264.9  


C:\Users\redde\AppData\Local\Temp\ipykernel_15296\2703295545.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  st_df = pd.read_sql('''


In [10]:
# 1 - team table
t_df = pd.read_sql('''
    select *
    from team
    limit 5;''', pg_conn)
print(t_df)

  teamid           teamname teamnameshort teamidpfr teamconference  \
0    ARI  Arizona Cardinals     Cardinals       CRD            NFC   
1    ATL    Atlanta Falcons       Falcons       ATL            NFC   
2    BAL   Baltimore Ravens        Ravens       RAV            AFC   
3    BUF      Buffalo Bills         Bills       BUF            AFC   
4    CAR  Carolina Panthers      Panthers       CAR            NFC   

  teamdivision teamconferencepre2002 teamdivisionpre2002  
0     NFC West                   NFC            NFC West  
1    NFC South                   NFC            NFC West  
2    AFC North                   AFC         AFC Central  
3     AFC East                   AFC            AFC East  
4    NFC South                   NFC            NFC West  


C:\Users\redde\AppData\Local\Temp\ipykernel_15296\2626329062.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  t_df = pd.read_sql('''


In [11]:
# 2 Part 1
q2_df = pd.read_sql('''
with totalcommissionscte as (
  select 
    customerid, 
    sum(commissionvalue) as totalcommissionpaid
  from betlog
  group by customerid
),
customercountcte as (
  select 
    count(customerid) as numofcustomers
  from totalcommissionscte
  where totalcommissionpaid > 20000
),
totalcustomerscte as (
  select 
    count(customerid) as totalcustomers
  from customer
)
select 
  numofcustomers, 
  totalcustomers, 
  (numofcustomers::decimal / totalcustomers) * 100 as percentageofcustomers
from customercountcte 
cross join totalcustomerscte;
''', pg_conn)
print(q2_df)

   numofcustomers  totalcustomers  percentageofcustomers
0             134            2000                    6.7


C:\Users\redde\AppData\Local\Temp\ipykernel_15296\1419285404.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  q2_df = pd.read_sql('''


In [12]:
# 2 Part 2
q2_df = pd.read_sql('''
with totalcommissionscte as (
  select 
    customerid, 
    sum(commissionvalue) as totalcommissionpaid
  from betlog
  group by customerid
)
select 
  customerfname, 
  customerlname, 
  totalcommissionpaid
from customer c 
join totalcommissionscte tc on c.customerid = tc.customerid
order by 
  totalcommissionpaid desc
limit 20;''', pg_conn)
print(q2_df)

   customerfname customerlname  totalcommissionpaid
0           Gary       McClain                95120
1        Matthew         Booth                89077
2         Vaughn         Ortiz                80355
3         Bailey        Sparks                71317
4          Nadia         Frank                71014
5          Alexa       Mendoza                70796
6         Violet        Tanner                68306
7          Simon         Cohen                60616
8        Harmoni       Burnett                59547
9          Lucas        Wagner                58778
10          Lara       Hoffman                58305
11         April        Martin                55734
12         Clark        Jordan                55683
13          Thea     Robertson                54413
14         Logan         Riley                53115
15        Warren       Barnett                53049
16      Bortimer       Fischer                52944
17       Adeline        Conner                52006
18        Au

C:\Users\redde\AppData\Local\Temp\ipykernel_15296\729413759.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  q2_df = pd.read_sql('''


In [13]:
# 3
q3_df = pd.read_sql('''
with numofbetsplacedcte as (
  select 
    customerid, 
    count(betid) as numofbetsplaced
  from betlog
  group by customerid
  having count(betid) >= 6
),
numofbetswoncte as (
  select 
    customerid, 
    count(betid) as numofbetswon, 
    sum(betamount) as totalamountwon
  from betlog
  where result = 'win'
  group by customerid
)
select 
  customerfname, 
  customerlname, 
  numofbetsplaced, 
  numofbetswon, 
  round(((numofbetswon::decimal / numofbetsplaced) * 100), 2) as winningpercentage, 
  totalamountwon
from customer c 
join numofbetsplacedcte bp on c.customerid = bp.customerid 
join numofbetswoncte bw on c.customerid = bw.customerid
order by 
  winningpercentage desc, 
  totalamountwon desc
limit 10;''', pg_conn)
print(q3_df)

  customerfname customerlname  numofbetsplaced  numofbetswon  \
0        Gianna        Palmer                7             7   
1        Milena        Fuller                6             6   
2          Esme         Lloyd               20            19   
3         Caleb      Reynolds               75            71   
4       Brinley        Pierce               23            21   
5         Avery      McKenzie                9             8   
6          Emma         Lewis               18            16   
7        Connor         Pitts                9             8   
8        Maddox       O'Brien                8             7   
9        Alayna          lark                8             7   

   winningpercentage  totalamountwon  
0             100.00           81000  
1             100.00             850  
2              95.00            2375  
3              94.67           56500  
4              91.30            2875  
5              88.89           45200  
6              88.89  

C:\Users\redde\AppData\Local\Temp\ipykernel_15296\4018389749.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  q3_df = pd.read_sql('''


In [14]:
# 4
q4_df = pd.read_sql('''
with numofbetsplacedcte as (
  select 
    customerid, 
    count(betid) as numofbetsplaced
  from betlog
  group by customerid
),
numofbetswoncte as (
  select 
    customerid, 
    count(betid) as numofbetswon, 
    sum(betamount) as amountcompanypaidout
  from betlog
  where result = 'win'
  group by customerid
),
numofbetslostcte as (
  select 
    customerid, 
    sum(betamount) as totalamountlost
  from betlog 
  where result = 'loss'
  group by customerid
),
commissionwoncte as (
  select 
    customerid, 
    sum(commissionvalue) as commissionwon
  from betlog
  group by customerid
),
companywoncte as (
  select 
    bl.customerid, 
    (totalamountlost + commissionwon) as amountcompanywon
  from numofbetslostcte bl 
  full outer join commissionwoncte comw on bl.customerid = comw.customerid
)
select 
  customerfname, 
  customerlname, 
  numofbetsplaced, 
  numofbetswon, 
  (amountcompanypaidout - amountcompanywon) as netamountlost
from customer c 
join numofbetsplacedcte bp on c.customerid = bp.customerid 
join numofbetswoncte bw on c.customerid = bw.customerid 
join companywoncte cw on c.customerid = cw.customerid
order by netamountlost desc 
limit 20;''', pg_conn)
print(q4_df)

   customerfname customerlname  numofbetsplaced  numofbetswon  netamountlost
0           Kate         Noble               84            48         152629
1          Tiana      Lawrence               60            44         124780
2           Owen      Garrison               45            27         119510
3            Lia         Drake               69            38         117425
4          Lucas         Eaton               90            56         113892
5          Royal          Bass               26            18          91086
6          Clark        Jordan               97            56          89142
7         Skylar         Terry               55            33          80441
8        Delilah          Bush               75            40          78391
9          Dante      Williams               14            10          66890
10        Summer         Smith               43            27          66594
11        Sophia      Robinson               64            43          63330

C:\Users\redde\AppData\Local\Temp\ipykernel_15296\1135762560.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  q4_df = pd.read_sql('''


In [15]:
# 5
q5_df = pd.read_sql('''
with gamerevenuecte as (
  select 
    g.gameid, 
    g.scheduleweek,
    sum(case when bl.result = 'loss' then bl.betamount else 0 end) as amountwon,
    sum(case when bl.result = 'win' then bl.betamount else 0 end) as amountlost
  from game g 
  left join betlog bl on g.gameid = bl.gameid
  where g.scheduleseason = 2023
  group by g.gameid, g.scheduleweek
),
gameresultcte as (
  select 
    gameid, 
    scheduleweek,
    case when amountwon > amountlost then 1 else 0 end as bookwinner,
    case when amountlost >= amountwon then 1 else 0 end as bookloser
  from gamerevenuecte
)
select 
  gr.scheduleweek, 
  count(gr.gameid) as gamesplayed, 
  sum(gr.bookwinner) as totalgameswon, 
  sum(gr.bookloser) as totalgameslost, 
  round((sum(gr.bookwinner)::decimal / count(gr.gameid)) * 100, 2) as percentwon,
  round((sum(gr.bookloser)::decimal / count(gr.gameid)) * 100, 2) as percentlost
from gameresultcte gr
group by gr.scheduleweek
order by 
  case 
    when gr.scheduleweek ~ '^\\d+$' then 1
    when gr.scheduleweek = 'Wildcard' then 2
    when gr.scheduleweek = 'Division' then 3
    when gr.scheduleweek = 'Conference' then 4
    when gr.scheduleweek = 'Superbowl' then 5
    else 6
  end,
  case 
    when gr.scheduleweek ~ '^\\d+$' then gr.scheduleweek::int
    else null
  end;''', pg_conn)
print(q5_df)

   scheduleweek  gamesplayed  totalgameswon  totalgameslost  percentwon  \
0             1           16              9               7       56.25   
1             2           16             10               6       62.50   
2             3           16             13               3       81.25   
3             4           16              9               7       56.25   
4             5           14             11               3       78.57   
5             6           15             12               3       80.00   
6             7           13              8               5       61.54   
7             8           16             11               5       68.75   
8             9           14             14               0      100.00   
9            10           14             10               4       71.43   
10           11           14             12               2       85.71   
11           12           16              9               7       56.25   
12           13          

C:\Users\redde\AppData\Local\Temp\ipykernel_15296\435572477.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  q5_df = pd.read_sql('''


In [16]:
# 6
q6_df = pd.read_sql('''
with homecount as (
  select t.teamid, count(g.gameid) as numathome
  from team t 
  left join game g on t.teamid = g.hometeamid and g.scheduleseason = 2023
  group by t.teamid
),
awaycount as (
  select t.teamid, count(g.gameid) as numaway
  from team t 
  left join game g on t.teamid = g.awayteamid and g.scheduleseason = 2023
  group by t.teamid
),
favoritecount as (
  select t.teamid, count(g.gameid) as numofbetsfor
  from team t 
  left join game g on t.teamid = g.teamfavoriteid and g.scheduleseason = 2023
  group by t.teamid
),
homewins as (
  select t.teamid, sum(case when g.homescore > g.awayscore and g.scheduleseason = 2023 then 1 else 0 end) as homewins
  from team t 
  left join game g on t.teamid = g.hometeamid
  group by t.teamid
),
awaywins as (
  select t.teamid, sum(case when g.awayscore > g.homescore and g.scheduleseason = 2023 then 1 else 0 end) as awaywins
  from team t 
  left join game g on t.teamid = g.awayteamid
  group by t.teamid
),
totalwins as (
  select t.teamid, (homewins + awaywins) as totalwins
  from team t 
  left join homewins hw on t.teamid = hw.teamid 
  left join awaywins aw on t.teamid = aw.teamid
),
totallosses as (
  select t.teamid, 
         (numathome + numaway - totalwins) as totallosses, 
         (numathome + numaway) as gamesplayed
  from team t 
  left join homecount hc on hc.teamid = t.teamid 
  left join awaycount ac on ac.teamid = t.teamid 
  left join totalwins tw on t.teamid = tw.teamid
),
homespread as (
  select t.teamid, sum(case when g.winner_line = 'home' and g.scheduleseason = 2023 then 1 else 0 end) as beatspreadathome
  from team t 
  left join game g on t.teamid = g.hometeamid
  group by t.teamid
),
awayspread as (
  select t.teamid, sum(case when g.winner_line = 'away' and g.scheduleseason = 2023 then 1 else 0 end) as beatspreadaway
  from team t 
  left join game g on t.teamid = g.awayteamid
  group by t.teamid
),
totalspread as (
  select t.teamid, (beatspreadathome + beatspreadaway) as totalbeatspread
  from team t 
  left join homespread hs on t.teamid = hs.teamid 
  left join awayspread aws on t.teamid = aws.teamid
)
select teamname, totalwins, totallosses, totalbeatspread, numofbetsfor, 
       (gamesplayed - numofbetsfor) as numofbetsagainst
from team t 
left join totalwins tw on t.teamid = tw.teamid 
left join totallosses tl on t.teamid = tl.teamid 
left join favoritecount fc on t.teamid = fc.teamid 
left join totalspread ts on t.teamid = ts.teamid
where t.teamid != 'PICK'
order by teamname asc;
''', pg_conn)
print(q6_df)

                 teamname  totalwins  totallosses  totalbeatspread  \
0       Arizona Cardinals          4           13                8   
1         Atlanta Falcons          7           10                5   
2        Baltimore Ravens         14            5               12   
3           Buffalo Bills         12            7                8   
4       Carolina Panthers          2           15                4   
5           Chicago Bears          7           10                8   
6      Cincinnati Bengals          9            8                8   
7        Cleveland Browns         11            7                9   
8          Dallas Cowboys         12            6               10   
9          Denver Broncos          8            9                6   
10          Detroit Lions         14            6               14   
11      Green Bay Packers         10            9               10   
12         Houston Texans         11            8               10   
13     Indianapolis 

C:\Users\redde\AppData\Local\Temp\ipykernel_15296\3520143150.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  q6_df = pd.read_sql('''


In [17]:
# Test Analysis

In [18]:
# 7
import statsmodels.api as sm
q7_df = pd.read_sql('''
with numofbetsplacedcte as (
  select 
    customerid, 
    count(betid) as numofbetsplaced
  from betlog
  group by customerid
),
numofbetswoncte as (
  select 
    customerid, 
    count(betid) as numofbetswon, 
    sum(betamount) as amountcompanypaidout
  from betlog
  where result = 'win'
  group by customerid
),
numofbetslostcte as (
  select 
    customerid, 
    sum(betamount) as totalamountlost
  from betlog 
  where result = 'loss'
  group by customerid
),
commissionwoncte as (
  select 
    customerid, 
    sum(commissionvalue) as commissionwon
  from betlog
  group by customerid
),
companywoncte as (
  select 
    bl.customerid, 
    (totalamountlost + commissionwon) as amountcompanywon
  from numofbetslostcte bl 
  full outer join commissionwoncte comw on bl.customerid = comw.customerid
)
select 
    c.customerid,
    customerage, 
    customertype, 
    customerincome, 
    householdsize,
    (amountcompanywon - amountcompanypaidout) as customervalue
from customer c 
join numofbetsplacedcte bp on c.customerid = bp.customerid 
join numofbetswoncte bw on c.customerid = bw.customerid 
join companywoncte cw on c.customerid = cw.customerid;''', pg_conn, index_col='customerid')
print(q7_df)

C:\Users\redde\AppData\Local\Temp\ipykernel_15296\3127772440.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  q7_df = pd.read_sql('''


            customerage customertype  customerincome  householdsize  \
customerid                                                            
1                    23       online          116000              1   
2                    41       online          111000              1   
3                    40       online           55000              1   
4                    43        local           32000              2   
5                    23        phone           46000              1   
...                 ...          ...             ...            ...   
1995                 43        phone           84000              4   
1996                 36        local           29000              2   
1998                 36        local           64000              1   
1999                 55        local           93000              1   
2000                 29        local          150000              4   

            customervalue  
customerid                 
1                  1

In [19]:
# create dummy variables
q7_df = pd.get_dummies(q7_df, columns=['customertype'], drop_first=True).astype(int)
print(q7_df)

            customerage  customerincome  householdsize  customervalue  \
customerid                                                              
1                    23          116000              1         102116   
2                    41          111000              1           1392   
3                    40           55000              1           -630   
4                    43           32000              2            122   
5                    23           46000              1           3473   
...                 ...             ...            ...            ...   
1995                 43           84000              4           1215   
1996                 36           29000              2            493   
1998                 36           64000              1           -695   
1999                 55           93000              1          -4923   
2000                 29          150000              4           -364   

            customertype_online  customertype_phon

In [20]:
# create correlation matrix
X = q7_df.drop('customervalue', axis=1)
y = q7_df['customervalue']
print(X.corr())
print('There are no multicollinearity issues between our independent variables. The highest correlation factor is -0.21 which is much smaller than we would typically worry about.')

                     customerage  customerincome  householdsize  \
customerage             1.000000       -0.000104      -0.004631   
customerincome         -0.000104        1.000000      -0.011312   
householdsize          -0.004631       -0.011312       1.000000   
customertype_online     0.012727       -0.010667       0.006797   
customertype_phone     -0.018372       -0.020110       0.012857   

                     customertype_online  customertype_phone  
customerage                     0.012727           -0.018372  
customerincome                 -0.010667           -0.020110  
householdsize                   0.006797            0.012857  
customertype_online             1.000000           -0.214191  
customertype_phone             -0.214191            1.000000  
There are no multicollinearity issues between our independent variables. The highest correlation factor is -0.21 which is much smaller than we would typically worry about.


In [21]:
# fit the model
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          customervalue   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     5.142
Date:                Fri, 25 Apr 2025   Prob (F-statistic):           0.000109
Time:                        07:14:41   Log-Likelihood:                -23256.
No. Observations:                1942   AIC:                         4.652e+04
Df Residuals:                    1936   BIC:                         4.656e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const               -4818.5672   4

In [22]:
model2_df = pd.read_sql('''
with numofbetsplacedcte as (
  select 
    customerid, 
    count(betid) as numofbetsplaced
  from betlog
  group by customerid
),
numofbetswoncte as (
  select 
    customerid, 
    count(betid) as numofbetswon, 
    sum(betamount) as amountcompanypaidout
  from betlog
  where result = 'win'
  group by customerid
),
numofbetslostcte as (
  select 
    customerid, 
    sum(betamount) as totalamountlost
  from betlog 
  where result = 'loss'
  group by customerid
),
commissionwoncte as (
  select 
    customerid, 
    sum(commissionvalue) as commissionwon
  from betlog
  group by customerid
),
companywoncte as (
  select 
    bl.customerid, 
    (totalamountlost + commissionwon) as amountcompanywon
  from numofbetslostcte bl 
  full outer join commissionwoncte comw on bl.customerid = comw.customerid
),
betsononeteam as (
select c.customerid, count(betid) as betsononeteam
from betlog bl join customer c on bl.customerid = c.customerid 
where beton not in ('over', 'under')
group by c.customerid, beton)
select 
    c.customerid,
    customerage, 
    customertype, 
    customerincome, 
    householdsize,
    (betsononeteam::decimal / numofbetsplaced) * 100 as percentononeteam,
    (amountcompanywon - amountcompanypaidout) as customervalue
from customer c 
join numofbetsplacedcte bp on c.customerid = bp.customerid 
join numofbetswoncte bw on c.customerid = bw.customerid 
join companywoncte cw on c.customerid = cw.customerid
join betsononeteam ot on c.customerid = ot.customerid;''', pg_conn)
model2_df = pd.get_dummies(model2_df, columns=['customertype'], drop_first=True).astype(int)
print(model2_df)

C:\Users\redde\AppData\Local\Temp\ipykernel_15296\3254213880.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  model2_df = pd.read_sql('''


       customerid  customerage  customerincome  householdsize  \
0             295           24          113000              2   
1              92           35           93000              1   
2             258           45          102000              2   
3            1786           51          154000              1   
4               7           39           25000              6   
...           ...          ...             ...            ...   
31066         737           43          159000              4   
31067         760           41           64000              4   
31068          75           37           92000              3   
31069        1620           43           99000              1   
31070         389           50           75000              2   

       percentononeteam  customervalue  customertype_online  \
0                     1           4739                    0   
1                     2           -188                    0   
2                     2       

In [23]:
print(model2_df.corr())

                     customerid  customerage  customerincome  householdsize  \
customerid             1.000000    -0.001372        0.098199      -0.029061   
customerage           -0.001372     1.000000       -0.014728       0.005866   
customerincome         0.098199    -0.014728        1.000000      -0.007826   
householdsize         -0.029061     0.005866       -0.007826       1.000000   
percentononeteam       0.023696     0.017410       -0.026045      -0.073714   
customervalue          0.044136     0.030338        0.093124       0.028059   
customertype_online    0.004956     0.040583       -0.026721      -0.005737   
customertype_phone     0.011693    -0.027123       -0.004643       0.015693   

                     percentononeteam  customervalue  customertype_online  \
customerid                   0.023696       0.044136             0.004956   
customerage                  0.017410       0.030338             0.040583   
customerincome              -0.026045       0.093124     

In [24]:
# fit the second model
X_2 = model2_df.drop('customervalue', axis=1)
y = model2_df['customervalue']
X_2, y = X_2.align(y, join='inner', axis=0)
X_2 = sm.add_constant(X_2)
model2 = sm.OLS(y, X_2).fit()
predictions2 = model2.predict(X_2)
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:          customervalue   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     207.0
Date:                Fri, 25 Apr 2025   Prob (F-statistic):          1.01e-301
Time:                        07:14:57   Log-Likelihood:            -3.7860e+05
No. Observations:               31071   AIC:                         7.572e+05
Df Residuals:                   31063   BIC:                         7.573e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                1108.3866   1

In [25]:
print('Adding in the percent of the bets that were on one team increased the r-squared to 0.045 from 0.013. While this is still not great for a model, it is better than the first one.')

Adding in the percent of the bets that were on one team increased the r-squared to 0.045 from 0.013. While this is still not great for a model, it is better than the first one.
